In [2]:
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [45]:
df = pd.read_csv('2024.csv')
df = df.rename(columns={'id': 'match_id'})
df

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,MatchReportURL,match_id,Home Score,Away Score,Winner
0,1.0,Fri,2023-08-11,20:00,Burnley,0.3,0–3,1.9,Manchester City,21572.0,Turf Moor,Craig Pawson,https://fbref.com/en/matches/3a6836b4/Burnley-...,0,0,3,Away
1,1.0,Sat,2023-08-12,12:30,Arsenal,0.8,2–1,1.2,Nott'ham Forest,59984.0,Emirates Stadium,Michael Oliver,https://fbref.com/en/matches/26a7f90c/Arsenal-...,1,2,1,Home
2,1.0,Sat,2023-08-12,15:00,Everton,2.7,0–1,1.5,Fulham,39940.0,Goodison Park,Stuart Attwell,https://fbref.com/en/matches/15addfc7/Everton-...,2,0,1,Away
3,1.0,Sat,2023-08-12,15:00,Sheffield Utd,0.5,0–1,1.9,Crystal Palace,31194.0,Bramall Lane,John Brooks,https://fbref.com/en/matches/55fd92c7/Sheffiel...,3,0,1,Away
4,1.0,Sat,2023-08-12,15:00,Brighton,4.0,4–1,1.5,Luton Town,31872.0,The American Express Community Stadium,David Coote,https://fbref.com/en/matches/56a137f7/Brighton...,4,4,1,Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,Sun,2024-05-19,16:00,Brentford,1.1,2–4,3.4,Newcastle Utd,17124.0,Gtech Community Stadium,Simon Hooper,https://fbref.com/en/matches/546e1a3d/Brentfor...,375,2,4,Away
376,38.0,Sun,2024-05-19,16:00,Chelsea,1.1,2–1,2.2,Bournemouth,39724.0,Stamford Bridge,Anthony Taylor,https://fbref.com/en/matches/7c034003/Chelsea-...,376,2,1,Home
377,38.0,Sun,2024-05-19,16:00,Crystal Palace,2.5,5–0,0.9,Aston Villa,25191.0,Selhurst Park,Darren Bond,https://fbref.com/en/matches/c975c7a6/Crystal-...,377,5,0,Home
378,38.0,Sun,2024-05-19,16:00,Liverpool,4.5,2–0,0.5,Wolves,60059.0,Anfield,Chris Kavanagh,https://fbref.com/en/matches/d4823ed5/Liverpoo...,378,2,0,Home


In [26]:
df = df.rename(columns={'id': 'match_id'})


In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm


# Initialize an empty DataFrame to store all results
agg_df = pd.DataFrame()

# Loop through match IDs; adjust range as needed for actual data processing
for link in tqdm(range(0, 380), desc="Processing Match IDs"):
    url = df.loc[link, 'MatchReportURL']

    # Fetch webpage
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve data for match ID {link}")
        continue
    
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.find_all('table')
    num_tables = len(tables)

    headers_list = []
    footer_list = []

    for i, table in enumerate(tables):
        head = table.find('thead')
        footer = table.find('tfoot')

        # Extract headers
        if head:
            headers = [header.text.strip() for header in head.find_all('th') if header.text.strip()]
            headers = [f'H{header}' if i < num_tables // 2 else f'A{header}' for header in headers]

        # Extract footer data
        if footer:
            footer_data = [cell.text.strip() for cell in footer.find_all('td') if cell.text.strip()]
            headers_list += headers[-len(footer_data):]
            footer_list += footer_data

    # Create sub DataFrame for the extracted data
    sub_df = pd.DataFrame([footer_list], columns=headers_list)
    sub_df['match_id'] = df.loc[link, 'match_id']

    # Concatenate to aggregate DataFrame
    agg_df = pd.concat([agg_df, sub_df], ignore_index=True)

    # Delay for a respectful scraping interval
    time.sleep(30)

agg_df


Processing Match IDs: 100%|██████████| 380/380 [3:13:25<00:00, 30.54s/it]  


,HMin,HGls,HAst,HPK,HPKatt,HSh,HSoT,HCrdY,HCrdR,HTouches,...,AInt,ATklW,APKwon,APKcon,AOG,ARecov,AWon,ALost,AWon%,match_id
0,989,0,0,0,0,6,1,0,1,496,...,4,12,0,0,0,54,13,13,50.0,0
1,990,2,2,0,0,15,7,2,0,902,...,8,10,0,0,0,34,20,12,62.5,1
2,990,0,0,0,0,19,9,0,0,531,...,4,4,0,0,0,43,14,9,60.9,2
3,990,0,0,0,0,8,1,3,0,441,...,4,9,0,0,0,55,31,13,70.5,3
4,990,4,2,1,1,26,11,2,0,769,...,9,12,0,1,0,41,21,12,63.6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,990,2,2,0,0,10,5,4,0,606,...,8,10,0,0,0,56,4,9,30.8,375
376,990,2,1,0,0,16,6,2,0,695,...,12,16,0,0,0,49,14,12,53.8,376
377,990,5,4,0,0,15,9,1,0,663,...,2,2,0,0,0,28,9,9,50.0,377
378,990,2,1,0,0,36,13,1,0,779,...,3,11,0,0,0,35,4,14,22.2,378


In [39]:
agg_df

,HMin,HGls,HAst,HPK,HPKatt,HSh,HSoT,HCrdY,HCrdR,HTouches,...,AInt,ATklW,APKwon,APKcon,AOG,ARecov,AWon,ALost,AWon%,match_id
0,989,0,0,0,0,6,1,0,1,496,...,4,12,0,0,0,54,13,13,50.0,0
1,990,2,2,0,0,15,7,2,0,902,...,8,10,0,0,0,34,20,12,62.5,1
2,990,0,0,0,0,19,9,0,0,531,...,4,4,0,0,0,43,14,9,60.9,2
3,990,0,0,0,0,8,1,3,0,441,...,4,9,0,0,0,55,31,13,70.5,3
4,990,4,2,1,1,26,11,2,0,769,...,9,12,0,1,0,41,21,12,63.6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,990,2,2,0,0,10,5,4,0,606,...,8,10,0,0,0,56,4,9,30.8,375
376,990,2,1,0,0,16,6,2,0,695,...,12,16,0,0,0,49,14,12,53.8,376
377,990,5,4,0,0,15,9,1,0,663,...,2,2,0,0,0,28,9,9,50.0,377
378,990,2,1,0,0,36,13,1,0,779,...,3,11,0,0,0,35,4,14,22.2,378


In [46]:
df = pd.merge(df, agg_df, on='match_id', how='left')

In [47]:
df

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,...,ACrs,AInt,ATklW,APKwon,APKcon,AOG,ARecov,AWon,ALost,AWon%
0,1.0,Fri,2023-08-11,20:00,Burnley,0.3,0–3,1.9,Manchester City,21572.0,...,15,4,12,0,0,0,54,13,13,50.0
1,1.0,Sat,2023-08-12,12:30,Arsenal,0.8,2–1,1.2,Nott'ham Forest,59984.0,...,6,8,10,0,0,0,34,20,12,62.5
2,1.0,Sat,2023-08-12,15:00,Everton,2.7,0–1,1.5,Fulham,39940.0,...,11,4,4,0,0,0,43,14,9,60.9
3,1.0,Sat,2023-08-12,15:00,Sheffield Utd,0.5,0–1,1.9,Crystal Palace,31194.0,...,17,4,9,0,0,0,55,31,13,70.5
4,1.0,Sat,2023-08-12,15:00,Brighton,4.0,4–1,1.5,Luton Town,31872.0,...,22,9,12,0,1,0,41,21,12,63.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,Sun,2024-05-19,16:00,Brentford,1.1,2–4,3.4,Newcastle Utd,17124.0,...,8,8,10,0,0,0,56,4,9,30.8
376,38.0,Sun,2024-05-19,16:00,Chelsea,1.1,2–1,2.2,Bournemouth,39724.0,...,20,12,16,0,0,0,49,14,12,53.8
377,38.0,Sun,2024-05-19,16:00,Crystal Palace,2.5,5–0,0.9,Aston Villa,25191.0,...,10,2,2,0,0,0,28,9,9,50.0
378,38.0,Sun,2024-05-19,16:00,Liverpool,4.5,2–0,0.5,Wolves,60059.0,...,5,3,11,0,0,0,35,4,14,22.2


In [48]:
from pathlib import Path

filepath = Path('final_2024.csv')  
df.to_csv(filepath, index=False)

In [35]:
from pathlib import Path

filepath = Path('agg_2024.csv')  
agg_df.to_csv(filepath, index=False)